# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

In [20]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages',engine)
df.shape

(19763, 39)

In [8]:
#prepare feature and classes as arrays
X = df['message'].values
Y = df[df.columns[4:39]].values

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [65]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer(smooth_idf=False, sublinear_tf=True, use_idf=True)),
        ('clf',MultiOutputClassifier(estimator=DecisionTreeClassifier(splitter='random', criterion='entropy')))
    ])     

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .30, random_state=42)

In [67]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000026F7FFDA168>)),
                ('tfidf',
                 TfidfTransformer(smooth_idf=False, sublinear_tf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                                        splitter='random')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [68]:
Y_pred=pipeline.predict(X_test)

In [69]:
f1_score=[]
precision=[]
classif=[]
for n in range(Y_pred.shape[1]):
    try:
        s=classification_report(Y_test[:,n], Y_pred[:,n],output_dict=True)
        classif.append(df.columns[n+4])
        precision.append(s['1.0']['precision'])
        f1_score.append(s['1.0']['f1-score'])
    except:
        print(df.columns[n+4]+' does not have a 1')
        continue
results=pd.DataFrame(classif,columns=['classifier'])
results['precision']=precision
results['f1-score']=f1_score
results['model']="DecisionTree"

print('avg_precision:'+str(sum(precision)/len(precision))+', avg_f1-score:'+str(sum(f1_score)/len(f1_score)))

avg_precision:0.14085071821931988, avg_f1-score:0.13226010020156034


In [70]:
results.to_excel("results_DecisionTree.xlsx", index=False)

In [71]:
results

,classifier,precision,f1-score,model
0,related,1.000000,1.000000,DecisionTree
1,request,0.425203,0.433346,DecisionTree
2,offer,0.000000,0.000000,DecisionTree
3,aid_related,0.564929,0.563593,DecisionTree
4,medical_help,0.101660,0.090994,DecisionTree
5,medical_products,0.043307,0.035200,DecisionTree
6,search_and_rescue,0.026490,0.022284,DecisionTree
7,security,0.000000,0.000000,DecisionTree
8,military,0.061798,0.049887,DecisionTree
9,water,0.095082,0.070388,DecisionTree


### 6. Improve your model
Use grid search to find better parameters. 

In [59]:
parameters = {
            'tfidf__smooth_idf': (True,False),
            'tfidf__sublinear_tf': (True,False),
            'tfidf__use_idf': (True,False)
}

scorers = {
    'precision_score': make_scorer(precision),
    'f1_score': make_scorer(f1_score)
}

In [60]:
cv = GridSearchCV(pipeline, parameters, return_train_score=scorers, cv=2, verbose=3)
cv.fit(X, Y)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True; total time= 3.4min
[CV 2/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=True; total time= 3.3min
[CV 1/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=False; total time= 3.7min
[CV 2/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=True, tfidf__use_idf=False; total time= 3.0min
[CV 1/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=False, tfidf__use_idf=True; total time= 3.4min
[CV 2/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=False, tfidf__use_idf=True; total time= 3.0min
[CV 1/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=False, tfidf__use_idf=False; total time= 3.4min
[CV 2/2] END tfidf__smooth_idf=True, tfidf__sublinear_tf=False, tfidf__use_idf=False; total time= 3.0min
[CV 1/2] END tfidf__smooth_idf=False, tfidf__sublinear_tf=True, tfidf__use_idf=True; total time= 3.4min
[CV 

AttributeError: 'list' object has no attribute '__name__'

AttributeError: 'list' object has no attribute '__name__'

In [63]:
pd.DataFrame(cv.cv_results_).to_excel("optimized_tfidf.xlsx", index=False)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [62]:
print(pd.DataFrame(cv.cv_results_))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0     115.694286     28.373690        86.006211       27.478485   
1     125.362668     38.281175        76.778524       15.991259   
2     117.579674     31.102848        76.124631       17.798345   
3     116.719483     30.711824        75.640467       16.453766   
4     115.659289     29.218122        75.961042       16.789986   
5     116.595871     29.492566        76.327291       16.301747   
6     116.158816     29.440580        76.006310       16.568397   
7     114.790281     29.484301        75.878617       16.648993   

  param_tfidf__smooth_idf param_tfidf__sublinear_tf param_tfidf__use_idf  \
0                    True                      True                 True   
1                    True                      True                False   
2                    True                     False                 True   
3                    True                     False                False   
4               